# LSTM
In this notebook the LSTM model will be implemented. We will use historical energy consumption data and meteorological data to make a prediction on future consumption values.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

seed=99
random.seed(seed)
np.random.seed(seed)

In [2]:
# Set up data
morocco = pd.read_csv("../data/processed/morocco_processed.csv")

In [3]:
morocco.set_index('DateTime', inplace=True)

In [4]:
display(morocco.head())
print('null values:', morocco.isnull().sum().sum())
print()
print('data types:', morocco.dtypes)

,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Consumption,year,month,day,hour
DateTime,,,,,,,,,,
2017-01-01 00:00:00,6.559,73.8,0.083,0.051,0.119,70425.53544,2017,1,1,0
2017-01-01 00:10:00,6.414,74.5,0.083,0.070,0.085,69320.84387,2017,1,1,0
2017-01-01 00:20:00,6.313,74.5,0.080,0.062,0.100,67803.22193,2017,1,1,0
2017-01-01 00:30:00,6.121,75.0,0.083,0.091,0.096,65489.23209,2017,1,1,0
2017-01-01 00:40:00,5.921,75.7,0.081,0.048,0.085,63650.44627,2017,1,1,0


null values: 0

data types: Temperature              float64
Humidity                 float64
Wind Speed               float64
general diffuse flows    float64
diffuse flows            float64
Consumption              float64
year                       int64
month                      int64
day                        int64
hour                       int64
dtype: object


### Literature review
In the previous thesis written by Andreas:  
* Scikit StandardScaler
* 48h lookback horizon
* Hyperparameter tuning on lookback horizon and number of layers using grid search.
* The parameters to adjust was selected so that the training would use less than 12h on a NVIDIA P100 graphics card.
* Did not use cross-validation given time series data. Instead used train-test split using the last 90 days for test. This makes sense, and is kinda how I selected my data.
* Batch size 32, max 150 epochs, learning rate schedulers on plateau, initial lr=0.001

## Split data into training and test sets
The Morocco dataset contains one year of data of 10 min granularity. In order to respect the temporal order of observations we'll split on a given timestep and use the data before the respective timestamp as training data and the data after will be used for testing.

In [5]:
split_date = pd.Timestamp('2017-12-01')

train = morocco[(morocco['month'] < 12)]
test = morocco[(morocco['month'] >= 12)]

X_train = train.drop(columns=['Consumption'])
y_train = train['Consumption']

X_test = test.drop(columns=['Consumption'])
y_test = test['Consumption']

NameError: name 'df' is not defined

In [6]:
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)

X_train: (48096, 9)
y_train: (48096,)


In [7]:
X_train.sample(5)

,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,year,month,day,hour
DateTime,,,,,,,,,
2017-09-08 17:50:00,25.58,64.65,0.259,254.000,35.740,2017,9,8,17
2017-08-02 21:30:00,25.67,59.40,4.903,0.073,0.126,2017,8,2,21
2017-09-27 18:30:00,22.45,80.20,4.921,28.620,27.360,2017,9,27,18
2017-07-13 06:00:00,21.92,76.50,4.919,2.311,1.882,2017,7,13,6
2017-04-28 13:40:00,23.50,41.31,0.087,884.000,55.770,2017,4,28,13


### Tensorflow implementation
You should have your data split into input sequences (X) and the corresponding labels (y). Here, X is expected to be a 3D array of shape (number of samples, time steps, features per step), and y is a 2D array of shape (number of samples, target variable).

Replace 50 with the number of LSTM units you want. The input_shape parameter should match the shape of your input data, excluding the sample dimension (e.g., (time steps, features per step)).

Here, X.shape[1] is the number of time steps, and X.shape[2] is the number of features per time step. Adjust the epochs and validation_split as necessary.

Assuming X_test is your test dataset prepared in the same way as your training dataset (X).

Remember, the effectiveness of your model heavily depends on the quality of your data, the way you've preprocessed it, and the hyperparameters of the model. It's often beneficial to experiment with different configurations, LSTM layers, and maybe adding dropout layers to improve the model's performance and prevent overfitting.

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
#from keras.preprocessing.sequence import TimeseriesGenerator

The LSTM model is defined with input_shape=(n_steps, n_features), which expects 3-dimensional input. Cannot directly pass X_train and y_train to the models fit() function without reshaping them to include the time steps dimension. You need to reshape your `X_train` and `X_test` data into the 3-dimensional shape expected by the LSTM layers. This is typically done by segmenting your time series data into sequences.  

Tensorflow introduces TimeseriesGenerator, which automatically handle the segmentation of time series data into (samples, time steps, features) format that LSTM layers expect.

In [ ]:
# Timeseriesgenerator is deprecated. Here is the manual code.
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

n_steps = lookback  # time steps parameter
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled.flatten(), n_steps)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled.flatten(), n_steps)


Selecting the window size/ n_steps. Pick one that allows the model to learn long-term dependencies, but hyptuning isn't neccessary. We perform short term forecasting, so we will use 2 days = 48 hours.

In [7]:
def scheduler(epoch, lr):
    if epoch < 100:
        return lr
    elif epoch%10==0:
        return lr * tf.math.exp(-0.3)
    else:
        return lr

def create_lstm_model(X_train, X_test, y_train, y_test, n_steps, n_features, n_epochs, n_layers):
    model = Sequential()
    for _ in range(n_layers - 1):
        model.add(LSTM(32, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    
    model.add(LSTM(32, activation='relu'))
    model.add(Dense(1))
    # model.compile(optimizer='adam', loss='mse')
    model.compile(optimizer=Adam(learning_rate = 0.001), loss='mse')
    
    callbacks = [tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)]
    
    #model.fit(train_generator, epochs=n_epochs, verbose=1, validation_data=test_generator, callbacks=callbacks)
    model.fit(X_train, y_train, epochs=n_epochs, verbose=1, callbacks=callbacks)
    return model

def fitting_and_eval(features, target, dataset_name, model_file_identifier='_', lookback = 6, n_layers = 2):
    # split_and_scale: Test set is the last 90 days
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=2160, random_state=seed, shuffle=False)
    
    # Scale X data
    XScaler = StandardScaler()
    XScaler.fit(X_train)
    X_train_scaled = XScaler.transform(X_train)
    X_test_scaled = XScaler.transform(X_test)

    # Scale Y data
    YScaler = StandardScaler()
    YScaler.fit(y_train.values.reshape(-1, 1))
    y_train_scaled = YScaler.transform(y_train.values.reshape(-1, 1))
    y_test_scaled = YScaler.transform(y_test.values.reshape(-1, 1))
    
    # Variables
    n_steps = lookback # hours
    batch = 64
    epochs = 150
    #generator_train = TimeseriesGenerator(scaled_X_train, scaled_y_train, length=n_steps, batch_size=batch)
    #generator_test = TimeseriesGenerator(scaled_X_test, scaled_y_test, length=n_steps, batch_size=batch)
    
    
    model_file_path = "lstm/" + model_file_identifier + "/" + dataset_name
    try: # Try to load the model if it already exists
        model = tf.keras.models.load_model(model_file_path)
    except: # If it doesn't exist, train the model and save it
        n_features = X_train.shape[1]
        model = create_lstm_model(X_train, X_test, y_train, y_test, n_steps, n_features, epochs, n_layers)
        model.save(model_file_path)
    
    # evaluate the model on the test set
    y_pred_scaled = model.predict(generator_test)
    y_pred = Yscaler.inverse_transform(y_pred_scaled)
    mape = mean_absolute_percentage_error(y_test[n_steps:], y_pred)
    mae = mean_absolute_error(y_test[n_steps:], y_pred)
    r2 = r2_score(y_test[n_steps:], y_pred)
    print(f'{dataset_name}, MAE: {mae:.2f}, MAPE: {mape:.3f}, R2: {r2:.3f}')

    return r2, mae, mape

In [8]:
%%time
data = morocco.dropna().reset_index(drop=True)
lookback=48 # 2 days
n_layers=2
model_file_identifier = str(lookback) + '_' + str(n_layers)

mape = fitting_and_eval(data.drop(columns=['Consumption', 'Temperature']), data['Consumption'], 'morocco', model_file_identifier=model_file_identifier, lookback=lookback, n_layers=n_layers)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/150


/Users/mariuslerstein/miniconda3/envs/master/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 8), dtype=float32). Expected shape (None, 6, 8), but input has incompatible shape (None, 8)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 8), dtype=float32)
  • training=True
  • mask=None

### Hyperparameter tuning
Bayesian search

In [ ]:
# Tuning done on 50 epochs 48 lagged steps
hyperparameter_tuning = {}
for lookback in [6, 12, 24]:
    for n_layers in [1, 2]:
        print(f'Hyperparameters: lookback={lookback}, n_layers={n_layers}')
        parameter_results = []
        for dataset_name in datasets_electricity:
            data = add_lagged_timesteps(datasets[dataset_name], lag_periods=[i for i in range(1, 49)], lagged_feature='Consumption').dropna().reset_index(drop=True)
            mape = lstm_fitting_and_evaluation(data.drop(columns=['DateTime', 'Consumption', 'Temperature']), data['Consumption'], dataset_name, model_file_identifier=str(lookback) + "_" + str(n_layers), lookback=lookback, n_layers=n_layers)
            parameter_results.append(mape)
        hyperparameter_tuning["Lookback:"+str(lookback) + ' ' + "Layers:"+str(n_layers)] = parameter_results

df = pd.DataFrame(hyperparameter_tuning,
                    index=[i for i in datasets_electricity.keys()])
df.plot.bar(figsize=(10, 5))
plt.title('MAPE for different hyperparameters with LSTM')
plt.ylabel('MAPE')
plt.xlabel('Dataset')
plt.show()

In [ ]:
# Visualize
df = pd.DataFrame(hyperparameter_tuning,
                    index=[i for i in datasets_electricity.keys()])
df.plot.bar(figsize=(10, 5))
plt.title('MAPE for different hyperparameters with LSTM')
plt.ylim(0, 0.1)
plt.ylabel('MAPE')
plt.xlabel('Dataset')
plt.show()

### PyTorch implementation
f you prefer not to use TensorFlow for building an LSTM model, another popular choice is PyTorch, a flexible deep learning framework that allows more explicit control over the model architecture and data flow. Here is a basic example of how to implement an LSTM for time series forecasting in PyTorch.

In PyTorch, you define your model as a class that extends nn.Module. Below is a simple example.

Your data should be formatted appropriately for PyTorch, typically as torch.Tensor objects. The input should be of shape (batch size, sequence length, number of features), and the labels should be of a compatible shape, depending on your output.

In [18]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [19]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
        # Define the output layer
        self.linear = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        
        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        
        # Index hidden state of last time step
        #

In [ ]:
# out[:, -1, :] just means we are taking the last LSTM output for each sequence
out = self.linear(out[:, -1, :]) 
return out

In [ ]:
# Prepare data
input_dim = 1  # number of features
hidden_dim = 50
num_layers = 2
output_dim = 1

model = LSTMModel(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, output_dim=output_dim)

# Mean Squared Error Loss
criterion = torch.nn.MSELoss()   

# Adam Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Training the model involves running the forward pass, calculating the loss, performing backpropagation, and updating the model parameters.

In this code, X_train and y_train should be your training data and labels, respectively, formatted as PyTorch tensors. Ensure that the shapes of your data match the expectations of the model (X_train should have three dimensions: batch, sequence, features).

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()  # Clear gradients w.r.t. parameters
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()  # Getting gradients w.r.t. parameters
    optimizer.step()  # Updating parameters
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

### Evaluate
After training, you can use the model to predict on new data. Ensure you format this data similarly to the training data before making predictions.

In [ ]:
model.eval()  # Set the model to evaluation mode
predictions = model(X_test)

Here, X_test should be your test dataset, prepared in the same way as your training dataset.

This example provides a basic introduction to implementing an LSTM in PyTorch for time series forecasting. Depending on your specific problem, you might need to adjust the model architecture, data preprocessing, or training process for optimal results.